<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/%E8%BC%95%E6%B2%B9%E6%A1%88%E5%AD%90_%E4%BA%A4%E5%8F%89%E9%A9%97%E8%AD%89_%E9%81%B8%E6%A8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
#!pip install catboost
from google.colab import drive
drive.mount('/content/drive')
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import warnings;warnings.simplefilter('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate
from sklearn.linear_model import BayesianRidge, LinearRegression,Lasso,Ridge
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,max_error,make_scorer
from math import sqrt
from sklearn.manifold import isomap
from numpy import sqrt
import plotly.express as px
import plotly

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# some function

In [0]:
def rmse(y,y_pred):
  return sqrt(mean_squared_error(y,y_pred))
rmse_scorer = make_scorer(rmse)

In [0]:
def metric(y_t,y_p,name):
  res = {'R2':r2_score(y_t,y_p),
         'RMSE':sqrt(mean_squared_error(y_t,y_p)),
         'mean_absolute_error':mean_absolute_error(y_t,y_p),
         'mean_absolute_error/y_mean' : mean_absolute_error(y_t,y_p) / np.mean(y_t),
         'RMSE/y_mean' : sqrt(mean_squared_error(y_t,y_p)) / np.mean(y_t),
         'y_mean' : np.mean(y_t),
         'std' : np.std(y_t),
         }
  return pd.DataFrame(res,index=[name])

In [0]:
def modeling(model,train,test,features_map,sequential=True):
  res = pd.DataFrame()
  y_train_pred = None
  y_test_pred = None
  init = True
  for y_name in features_map.keys():
    
    # if sequential?
    if (sequential == True) and (init == False):
      X_train = np.hstack([np.array(X_train),np.array(y_train_pred)])
      X_test  = np.hstack([np.array(X_test),np.array(y_test_pred)])
    else:
      X_train = train[features_map[y_name]]
      X_test = test[features_map[y_name]]

    #fit
    model.fit(X_train,train[y_name])
    
    #predict
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    #to dataframe
    y_train_pred = pd.DataFrame(y_train_pred,columns=[y_name])
    y_test_pred = pd.DataFrame(y_test_pred,columns=[y_name])
    
    # add to res
    res = res.append(metric(test[y_name],y_test_pred[y_name],y_name))
    
    #
    init = False
  res.loc['AVG'] = res.mean()
  return res

# load data

In [94]:
data = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/test_250.csv',index_col=0)
train = data.loc[data.index %5 != 0]
test = data.loc[data.index %5 == 0]
test = test.drop(95,axis=0)
print(train.shape)
print(test.shape)

(200, 27)
(49, 27)


In [0]:
#train_4521 = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/train_4521.csv',index_col=0)
#train = train.append(train_4521)
#print(train.shape)
#print(test.shape)

#  clean data

In [96]:
def clean(df):
  condition=(df['N+A']>=36)
  return df.loc[condition]
#==========================
train = clean(train)
test = clean(test)
print(train.shape)
print(test.shape)

(181, 27)
(45, 27)


# 定義欄位

In [0]:
cols = train.columns.tolist()
features = cols[:4]
y_columns = cols[4:]

# 特徵縮放

In [0]:
scaler_x = StandardScaler()
train[features] = scaler_x.fit_transform(train[features])
test[features] = scaler_x.transform(test[features])

# 降維萃取特徵

In [99]:
for i,t in zip(['f1','f2','f3'],['T10','T50','T90']):
  iso = isomap.Isomap(n_components=1)
  train[i] = iso.fit_transform(train[[t,'N+A']]) 
  test[i] = iso.transform(test[[t,'N+A']]) 
  features = features + [i]
print(features)
print(y_columns)

['T10', 'T50', 'T90', 'N+A', 'f1', 'f2', 'f3']
['C5NP', 'C5IP', 'C5N', 'C6NP', 'C6IP', 'C6N', 'C6A', 'C7NP', 'C7IP', 'C7N', 'C7A', 'C8NP', 'C8IP', 'C8N', 'C8A', 'C9NP', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N', 'C10A']


# 化工domain特徵

In [100]:
for df in [train,test]:
  df['f4'] = df['T90']-df['T50']
  df['f5'] = df['T50']-df['T10']
  df['f6'] = df['T90']-df['T10']
features = features+['f4','f5','f6']
print(features)
print(y_columns)

['T10', 'T50', 'T90', 'N+A', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6']
['C5NP', 'C5IP', 'C5N', 'C6NP', 'C6IP', 'C6N', 'C6A', 'C7NP', 'C7IP', 'C7N', 'C7A', 'C8NP', 'C8IP', 'C8N', 'C8A', 'C9NP', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N', 'C10A']


# 平方特徵

In [101]:
for df in [train,test]:
  df['f7'] = df['T10']**2
  df['f8'] = df['T50']**2
  df['f9'] = df['T90']**2
  df['f10'] = df['N+A']**2
features = features+['f7','f8','f9','f10']
print(features)
print(y_columns)

['T10', 'T50', 'T90', 'N+A', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10']
['C5NP', 'C5IP', 'C5N', 'C6NP', 'C6IP', 'C6N', 'C6A', 'C7NP', 'C7IP', 'C7N', 'C7A', 'C8NP', 'C8IP', 'C8N', 'C8A', 'C9NP', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N', 'C10A']


# 特徵篩選

In [0]:
def features_selected(select=True):
  features_map = {}
  if select == True:
    for y_name in y_columns:
      corrs = abs(train[features+[y_name]].corr())[[y_name]].drop(y_name).sort_values(by=y_name,ascending=False)
      best_feature = corrs.head(1).index.tolist()[0]
      if best_feature not in ['T10','T50','T90','N+A']:
        features_map[y_name] =  ['T10','T50','T90','N+A'] + [best_feature]
      else:
        features_map[y_name] =  ['T10','T50','T90','N+A']
  else:
    for y_name in y_columns:
      features_map[y_name] = features
  return features_map

In [103]:
features_map = features_selected(select=True)
res = pd.DataFrame()
for y_name in y_columns:
  res.loc[y_name,'features'] = str(features_map[y_name])
res

,features
C5NP,"['T10', 'T50', 'T90', 'N+A']"
C5IP,"['T10', 'T50', 'T90', 'N+A']"
C5N,"['T10', 'T50', 'T90', 'N+A']"
C6NP,"['T10', 'T50', 'T90', 'N+A', 'f2']"
C6IP,"['T10', 'T50', 'T90', 'N+A']"
C6N,"['T10', 'T50', 'T90', 'N+A']"
C6A,"['T10', 'T50', 'T90', 'N+A']"
C7NP,"['T10', 'T50', 'T90', 'N+A', 'f5']"
C7IP,"['T10', 'T50', 'T90', 'N+A', 'f5']"
C7N,"['T10', 'T50', 'T90', 'N+A', 'f3']"


# 交叉驗證選模

In [104]:
model = CatBoostRegressor(learning_rate = 0.1,n_estimators=100,verbose=0)
res = pd.DataFrame()
for y_name in features_map.keys():
  cv = cross_validate(model,train[features_map[y_name]],train[y_name],cv=5,scoring=rmse_scorer)
  res.loc[y_name,'五次交叉驗證_RMSE'] = str(cv['test_score'])
  res.loc[y_name,'RMSE_AVG'] = cv['test_score'].mean()
res.loc['AVG'] = res.mean()
res.style.background_gradient()

,五次交叉驗證_RMSE,RMSE_AVG
C5NP,[1.01626784 1.30412294 1.45545602 0.48856705 0.75314188],1.00351
C5IP,[0.94725394 0.9496485 0.65278537 0.48989801 0.51663561],0.711244
C5N,[0.10184946 0.16027331 0.09079552 0.08012125 0.11906937],0.110422
C6NP,[0.71968815 0.87479853 0.83846288 0.58142012 0.73341473],0.749557
C6IP,[0.62167143 1.07017941 0.70106501 0.70424487 0.74796677],0.769025
C6N,[0.93850683 1.61171502 1.20145444 1.47979852 1.20259285],1.28681
C6A,[1.03211187 1.30207293 1.29303273 0.23070308 0.26704653],0.824993
C7NP,[0.67306512 0.93098442 0.55402569 0.76039339 0.46798101],0.67729
C7IP,[0.77761513 0.8725739 0.96910278 0.56107413 0.73680856],0.783435
C7N,[2.37260212 2.79685016 2.40811423 1.20562507 1.73313755],2.10327


In [105]:
model = SVR()
res = pd.DataFrame()
for y_name in features_map.keys():
  cv = cross_validate(model,train[features_map[y_name]],train[y_name],cv=5,scoring=rmse_scorer)
  res.loc[y_name,'五次交叉驗證_RMSE'] = str(cv['test_score'])
  res.loc[y_name,'RMSE_AVG'] = cv['test_score'].mean()
res.loc['AVG'] = res.mean()
res.style.background_gradient()

,五次交叉驗證_RMSE,RMSE_AVG
C5NP,[1.18892417 1.56698087 1.44870306 0.58408767 0.59514594],1.07677
C5IP,[1.34296115 1.11623359 0.78240638 0.49054768 0.55661316],0.857752
C5N,[0.11050909 0.16685531 0.08855373 0.09923334 0.09841419],0.112713
C6NP,[0.70475476 0.88514024 0.900819 0.74478832 0.73602978],0.794306
C6IP,[0.76881327 1.13299984 0.6656543 0.75887243 0.56865769],0.779
C6N,[1.11095164 1.831741 1.09552374 1.90909737 1.20051438],1.42957
C6A,[1.03745909 0.74425924 1.25109653 0.18345016 0.23000143],0.689253
C7NP,[0.68772782 0.96690753 0.55138889 0.7264716 0.42869298],0.672238
C7IP,[0.8858164 1.05604822 1.16201268 0.80309613 0.58785579],0.898966
C7N,[2.16213501 3.44483401 3.3719455 1.96441219 1.75615261],2.5399


In [106]:
model = LinearRegression()
res = pd.DataFrame()
for y_name in features_map.keys():
  cv = cross_validate(model,train[features_map[y_name]],train[y_name],cv=5,scoring=rmse_scorer)
  res.loc[y_name,'五次交叉驗證_RMSE'] = str(cv['test_score'])
  res.loc[y_name,'RMSE_AVG'] = cv['test_score'].mean()
res.loc['AVG'] = res.mean()
res.style.background_gradient()

,五次交叉驗證_RMSE,RMSE_AVG
C5NP,[1.430854 1.61796747 1.43935026 0.70084535 0.75350342],1.1885
C5IP,[1.3252714 1.15805413 0.95017532 0.8030512 0.73566175],0.994443
C5N,[0.10142388 0.14299291 0.09424221 0.09797296 0.1123282 ],0.109792
C6NP,[0.89494726 1.0641104 1.07613588 0.81303564 0.91997656],0.953641
C6IP,[0.59109962 0.89230136 0.95440234 0.83400715 0.73925192],0.802212
C6N,[1.13341655 1.92009866 1.28730029 1.5851199 1.25223521],1.43563
C6A,[1.19963214 0.99200864 1.36116025 0.42461419 0.24220668],0.843924
C7NP,[0.87746639 1.04147019 0.80314499 0.7583638 0.5595625 ],0.808002
C7IP,[1.57458989 1.80874791 1.37095859 1.42980741 1.00940044],1.4387
C7N,[2.48008247 3.40566033 3.02478942 2.28032991 1.63805538],2.56578


# 測試49筆

In [0]:
def mape(a, b): 
    mask = a != 0
    return (np.fabs(a - b)/a)[mask].mean()
def metric(y_t,y_p,name):
  res = {'R2':r2_score(y_t,y_p),
         'RMSE':sqrt(mean_squared_error(y_t,y_p)),
         'MAPE':mape(y_t,y_p),
         }
  return pd.DataFrame(res,index=[name])

In [0]:
model = CatBoostRegressor(learning_rate=0.1,n_estimators=100,verbose=False)

In [109]:
y_pred = pd.DataFrame(index=test.index,columns=y_columns)
for y_name in y_columns:
  model.fit(train[features_map[y_name]],train[y_name])
  y_pred[y_name] = model.predict(test[features_map[y_name]])
y_pred.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
5,0.870471,0.593798,0.135206,1.793178,1.345332,4.503771,0.424418,6.366381,5.843314,19.002213,6.700151,6.915043,9.248758,9.142140,7.122469,3.551796,6.212740,3.462967,2.165126,0.517840,1.940315,0.267635,0.403522
10,1.253746,0.750253,0.133204,2.156297,1.758642,4.012801,0.528990,7.103729,6.925730,18.685251,6.434458,7.605884,11.707909,7.758975,7.694392,3.546038,6.619926,2.693356,1.977313,0.622541,1.727541,0.130807,0.393819
15,9.908933,6.041595,0.916467,7.465614,8.865948,8.251419,2.577272,5.664464,6.800451,12.305634,4.298214,4.258599,6.252503,4.835193,2.882519,1.643111,3.657829,1.494510,0.686722,0.229564,0.855177,0.089796,0.167184
20,11.123202,7.461634,0.984311,7.005910,8.097373,9.554501,3.428797,5.327828,5.199315,11.409744,4.493293,3.929704,4.668469,4.519977,3.552025,1.884260,3.057213,1.556445,0.797737,0.345136,0.955334,0.084453,0.167539
25,9.357260,8.332247,0.974443,5.407562,7.786091,6.994137,1.549924,4.766116,5.380522,10.558355,3.658319,4.187675,5.383117,4.252552,4.208348,2.691316,4.720281,2.540818,1.744624,1.401234,2.794369,0.290963,0.876345


In [110]:
res = pd.DataFrame()
for y_name in y_columns:
  res = res.append(metric(test[y_name],y_pred[y_name],y_name))
res.loc['AVG'] = res.mean()
res.style.background_gradient()

,R2,RMSE,MAPE
C5NP,0.980687,0.521096,0.779937
C5IP,0.963896,0.586444,1.2631
C5N,0.961623,0.0624133,0.613573
C6NP,0.892492,0.616514,0.382144
C6IP,0.924112,0.716373,0.604798
C6N,0.83634,0.768337,0.0890505
C6A,0.740345,0.466157,0.573097
C7NP,0.665914,0.469489,0.0609258
C7IP,0.757121,0.710169,0.0925627
C7N,0.940859,1.41755,0.0629066


# count loss

In [111]:
se = (y_pred[y_columns] - test[y_columns])**2
mse = se.mean(axis=1)
rmse = sqrt(mse)
test['loss'] = rmse
test[['loss']].head()

,loss
5,1.230493
10,1.540121
15,0.793303
20,0.420537
25,0.251481


# plot

In [0]:
iso_x = isomap.Isomap(n_components=1)
iso_y = isomap.Isomap(n_components=1)
test['iso_x'] = iso_x.fit_transform(test[['T10','T50','T90']])
test['iso_y'] = iso_y.fit_transform(test[['f4','f5','f6']])

In [113]:
test[['T10','T50','T90','N+A']] = scaler_x.inverse_transform(test[['T10','T50','T90','N+A']])
fig = px.scatter(test,x="iso_x",y="iso_y",
                 color="N+A",
                 size='loss')
fig.show()

# to html

In [114]:
plotly.offline.plot(fig,filename = 'oil.html', auto_open=False)

'oil.html'